# Pregunta 2

A continuación se importan algunas librerías estándar de ``python`` que serán de utilidad. Luego se definen funciones auxiliares que serán usadas para

In [265]:
# Standard library
from math import ceil
from random import randint, randrange
from time import time

# Auxiliary functions

# Fast exponentiation
def _fast_exp(a, b):
    if not b:
        return 1
    result = 1
    power = a
    while b:
        if b % 2:
            result = result * power
        b = b // 2
        power = power * power
    return result

# Fast modular exponentiation
def _fast_exp_mod(a, b, n):
    if not b:
        return 1
    if b:
        result = 1
        power = a
        while b:
            if b % 2:
                result = (power * result) % n
            b = b // 2
            power = (power * power) % n
        return result
    return _fast_exp_mod(_mod_inverse(a, n), -b, n)

# Modular inverse
def _mod_inverse(a, n):
    r, s, t = _ext_euclidean(a, n)
    return s % n

# Greatest common divisor
def _gcd(a, b):
    while b:
        temp = b
        b = a % b
        a = temp
    return a

# Extended Euclidean algorithm
def _ext_euclidean(a, b):
    r_0 = a
    s_0 = 1
    t_0 = 0
    r_1 = b
    s_1 = 0
    t_1 = 1
    while r_1:
        r_2 = r_0 % r_1
        s_2 = s_0 - (r_0 // r_1) * s_1
        t_2 = t_0 - (r_0 // r_1) * t_1
        r_0 = r_1
        s_0 = s_1
        t_0 = t_1
        r_1 = r_2
        s_1 = s_2
        t_1 = t_2
    return r_0, s_0, t_0

# Check if n is a power of another number (optimized considering n > 3)
def _is_power(n):
    k = 2
    lim = 4
    while lim <= n and k < 25:
        if _has_integer_root(n, k):
            return True
        k = k + 1
        lim = lim * 2
    return False

# Check if n is a k-th degree power (optimized considering n > 3)
def _has_integer_root(n, k):
    a = 1
    while _fast_exp(a, k) < n:
        a = a * 2
    return _has_integer_root_interval(n, k, a // 2, a)

# Check if n is a k-th degree power inside an interval
def _has_integer_root_interval(n, k, i, j):
    while i <= j:
        if i == j:
            return n == _fast_exp(i, k)
        p = (i + j) // 2
        val = _fast_exp(p, k)
        if n == val:
            return True
        elif val < n:
            i = p + 1
        else:
            j = p - 1
    return False

# Primality test (Solovay – Strassen) with some slight optimizations
def _primality_test(n, k):
    if _is_power(n):
        return False
    neg = 0
    for _ in range(k):
        a = randint(2, n - 1)
        if _gcd(a, n) > 1:
            return False
        b = _fast_exp_mod(a, (n - 1) // 2, n)
        if b == n - 1:
            neg = neg + 1
        elif b != 1:
            return False
    return neg > 0

# Obtain all primes smaller or equal to n (Sieve of Eratosthenes)
def _get_primes(n):
    prime_idx = [1] * (n + 1)
    p = 2
    while (p * p) <= n:
        if prime_idx[p]:
            for i in range(p * p, n + 1, p):
                prime_idx[i] = 0
        p = p + 1
    return [p for p in range(2, n + 1) if prime_idx[p]]

# Generate a pair of unique random prime numbers in the interval [a,b]
def _generate_primes(a, b):
    # Number of repetitions for the primality test
    k = 34  # Probability of error is (1/2)^k

    # Trivial case
    if a == 2:
        return [2, 3]

    # Pre-calculate first few primes
    first_primes = _get_primes(10000)

    # Search for primes by discarding most candidates in constant time
    primes = []
    while True:
        p = randrange(a + 1, b + 1, 2)  # Only check odd numbers
        if p in primes:  # Both primes need to be different
            continue
        candidate = True
        for n in first_primes:  # Discard with first n prime factors
            if p <= n:
                break
            if not (p % n):
                candidate = False
                break
        if candidate and _primality_test(p, k):  # Test for primality
            primes.append(p)
            if len(primes) == 2:
                return primes

# Generate a random number that is co-prime to n
def _generate_coprime(n):
    # Trivial case
    if n == 2:
        return 1

    # Search for co-primes
    while True:
        c = randint(2, n - 1)
        if _gcd(c, n) == 1:  # GCD must be 1
            return c

A

In [266]:
# RSA

# Receiver
class RSAReceiver:
    def __init__(self, bit_len):
        """
        Arguments:
            bit_len: A lower bound for the number of bits of N,
            the second argument of the public and secret key.
        """
        # Generate random primes P,Q
        p, q = _generate_primes(2 ** (ceil(bit_len / 2) - 1), 2 ** (ceil(bit_len / 2)) - 1)

        # Define N and phi(N)
        n = p * q
        phi_n = n - p - q + 1

        # Generate random co-prime d (to phi(N))
        d = _generate_coprime(phi_n)

        # Get the modular inverse e for d in mod phi(N)
        e = _mod_inverse(d, phi_n)

        # Define private & public keys
        self.private_key = (d, n)
        self.public_key = (e, n)

    # Get public key
    def get_public_key(self):
        """
        Returns:
            public_key: Public key expressed as a Python 'bytearray' using the
            PEM format. This means the public key is divided in:
            (1) The number of bytes of e (4 bytes)
            (2) The number e (as many bytes as indicated in (1))
            (3) The number of bytes of N (4 bytes)
            (4) The number N (as many bytes as indicated in (3))
        """
        e_bytes = ceil(self.public_key[0].bit_length() / 8)
        pem_key = bytearray(e_bytes.to_bytes(4, 'big'))
        pem_key += self.public_key[0].to_bytes(e_bytes, 'big')
        n_bytes = ceil(self.public_key[1].bit_length() / 8)
        pem_key += bytearray(n_bytes.to_bytes(4, 'big'))
        pem_key += self.public_key[1].to_bytes(n_bytes, 'big')
        return pem_key

    # Decrypt the cipher
    def decrypt(self, ciphertext):
        """
        Arguments:
            ciphertext: The ciphertext to decrypt.
        Returns:
            message: The original message.
        """
        # TODO: decrypt blocks using private key
        return ''

# Sender
class RSASender:
    def __init__(self, public_key):
        """
        Arguments:
            public_key: The public key that will be used to encrypt messages.
        """
        self.receiver_key = public_key

    # Encrypt the message
    def encrypt(self, message):
        """
        Arguments:
            message: The plaintext message to encrypt.
        Returns:
            ciphertext: The encrypted message.
        """
        # TODO: encrypt blocks using receiver key
        return ''

A

In [267]:
# Test RSA protocol
if __name__ == '__main__':
    receiver = RSAReceiver(2048)
    print(receiver.get_public_key())


bytearray(b'\x00\x00\x01\x00B\x02\xdfYu\x9evi\x83 W\x8c\xce\xef\x98wY[\xc6\xc1\x94[d$\xf8\xdd\x93WolG\x01\x04\x12\x83\xbb|\x15\xc2\x1d\x15\xa2\x90\x9a\xdf\xe6\x9b6K-\xf2\x1d\xd2\xc4\xc7\x9a\x15*\xa906n\x039\xa0m\x12\xdcj\x04Mnh,g\xd8\xa25k%\xfdz\xed\xd0\xbb\xa4\xc9\xfef.\xae\xe3y\x14\xff\'\xac\xe8\x19\xbc\xd1\x1e\xd1$\\12\x1a\xc5\xc8\x80\x87\xf6,\xbf\xc5\xb6\x1e\t}\x10h\x02\x18\xf1\x18\xbam:\xb5\xe4u\xe5pnm\xad\x18\xf5\x98\xd7\x91m\xe1x\xd5\xf2&J%\xa7\x01!\xb76}\xe9\x8e\x95\xe7\x9eM8\xae\xdcO\r\x98k+.\xc8\xef\x7f?%\xb8\xf7\xa6\xdd\xa8\x94\xb6m\xd8\xeb<`n\x82\\\xa8<\x0e\xb2\x1b+\x1a4\x18\xa4\x97\x19\xaa\xd3(\xa6\x1f"W\x1d\xbf\x1a\xd8>3\xac\x01\xcd\x93\xbb?PC:D\x86\x94\n`z\xee\x99k\x06\x01\x08\xf1\x04PFNDQ\x88 \xd0\xd2\t\x0b\xd5\xe9D\x13\xe5-\x00\x00\x01\x00\xa2\xc80\xf2>\xe5s\x95[\xcc\xfa\xdb\x81D\xd1\xf0\xf5\x98\xef\xd2\xbb&\x83A\xfe\xa8H\xfb\xae\xc7\xff\xa1v\xca\xf9z\xe7\xc6\x0bE\xca\xcb+g\x0c\x89-\xd5\xa4\x1f\x9cK\x06\xc2a\xb5\xc62\x1c\xe6\xe4\xe9dI\xec\xaa\x05#\xad\xc4\x9c\xc1\xfbVG